In [2]:
from pathlib import Path
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

2024-06-20 07:42:12.765156: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 07:42:12.804484: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 07:42:13.631428: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 07:42:16.790572: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 07:42:24.639736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [3]:
def load_pose_data(path: Path):
    data = []
    with open(path, "r") as f:
        for line in f:
            coordinates = []
            for coord_str in line.strip().split():
                x, y, z = map(float, coord_str.split(","))
                coordinates.append([x, y, z])
            data.append(coordinates)

    matrix = np.array(data)
    return matrix

In [5]:
data_path = Path('data/pose')
pose_file_paths = list(data_path.glob('*.dat'))
pose_file_paths.sort(key=lambda p: p.stem)
pose_file_paths

[PosixPath('data/pose/00_none.dat'),
 PosixPath('data/pose/01_right.dat'),
 PosixPath('data/pose/02_left.dat'),
 PosixPath('data/pose/03_horizontal.dat'),
 PosixPath('data/pose/04_vertical1.dat'),
 PosixPath('data/pose/05_vertical2.dat'),
 PosixPath('data/pose/06_ok.dat'),
 PosixPath('data/pose/07_bad.dat'),
 PosixPath('data/pose/08_y.dat'),
 PosixPath('data/pose/09_m.dat'),
 PosixPath('data/pose/10_c.dat'),
 PosixPath('data/pose/11_a.dat')]

In [6]:
samples = [(path.stem, load_pose_data(path)) for path in pose_file_paths]

In [12]:
labels = [label for label, _ in samples]
labels

['00_none',
 '01_right',
 '02_left',
 '03_horizontal',
 '04_vertical1',
 '05_vertical2',
 '06_ok',
 '07_bad',
 '08_y',
 '09_m',
 '10_c',
 '11_a']

In [9]:
dataset = [(result, i) for i, (_, results) in enumerate(samples) for result in results]

Type:        list
String form:
[(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
           [ 4.69088140e-03, -3.2138310 <...>  -1.2387065e+00,  2.7142397e-01],
           [-9.9758710e-02, -1.3284926e+00,  1.2514532e-01]]), 11)]
Length:      24000
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.

In [10]:
x = np.array([data for data, _ in dataset])
y = np.array([label for _, label in dataset])

In [13]:
input_shape = (x.shape[1], x.shape[2])
label_len = len(labels)

In [14]:
keras_model = keras.Sequential([
    keras.layers.InputLayer(input_shape, name='input'),
    keras.layers.Flatten(name='Flatten'),
    keras.layers.Dense(64, activation='elu', name='Dense'),
    keras.layers.Dropout(0.2, name='Dropout'),
    keras.layers.Dense(label_len, activation='softmax', name='Output'),
])

In [15]:
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.8)

In [18]:
keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = keras_model.fit(train_x, train_y, epochs=10, validation_split=0.2)
history

Epoch 1/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 1s 838us/step - accuracy: 1.0000 - loss: 2.5549e-05 - val_accuracy: 1.0000 - val_loss: 4.5635e-09
Epoch 2/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - accuracy: 1.0000 - loss: 2.3798e-07 - val_accuracy: 1.0000 - val_loss: 2.7940e-10
Epoch 3/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 1.0000 - loss: 7.7181e-08 - val_accuracy: 1.0000 - val_loss: 1.2418e-10
Epoch 4/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 1.0000 - loss: 1.2999e-07 - val_accuracy: 0.9919 - val_loss: 0.0158
Epoch 5/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9999 - loss: 4.7356e-04 - val_accuracy: 1.0000 - val_loss: 1.4280e-09
Epoch 6/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 1.0000 - loss: 3.2376e-06 - val_accuracy: 1.0000 - val_loss: 9.5335e-08
Epoch 7/10
480/480 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 1.0000 - loss: 1.1330e-04 - val_accuracy: 1.0000 - val_loss: 3.1044e-10
Epoch 8/10
480/480 ━━━━━━━━━━━━━━━━━━━

In [19]:
keras_model.evaluate(test_x, test_y)

150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 535us/step - accuracy: 1.0000 - loss: 2.0270e-09


[2.4835122847122193e-09, 1.0]

In [20]:
keras_model.export('models/pose')

INFO:tensorflow:Assets written to: models/pose/assets


INFO:tensorflow:Assets written to: models/pose/assets


Saved artifact at 'models/pose'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 31, 3), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  140084128041088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140084128044960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140084128045664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140084128046192: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [21]:
conv = tf.lite.TFLiteConverter.from_saved_model('models/pose')
tflite_model = conv.convert()
with open('models/pose.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1718870284.151250    6987 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1718870284.151280    6987 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-06-20 07:58:04.153954: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/pose
2024-06-20 07:58:04.156252: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-20 07:58:04.156259: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: models/pose
2024-06-20 07:58:04.161388: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-20 07:58:04.161757: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-06-20 07:58:04.180960: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: models/pose
2024-06-20 07:58:04.184461: I tensorflow/cc/saved_model/loader.cc:317] SavedMode